(In order to load the stylesheet of this notebook, execute the last code cell in this notebook)

# Recommender System for Amazon Electronics

In this assignment, we will be working with the [Amazon dataset](http://cs-people.bu.edu/kzhao/teaching/amazon_reviews_Electronics.tar.gz). You will build a recommender system to make predictions related to reviews of Electronics products on Amazon.

Your grades will be determined by your performance on the predictive tasks as well as a brief written report about the approaches you took.

This assignment should be completed **individually**.

## Files

**train.json** 1,000,000 reviews to be used for training. It is not necessary to use all reviews for training if doing so proves too computationally intensive. The fields in this file are:

* **reviewerID** The ID of the reviewer. This is a hashed user identifier from Amazon.

* **asin** The ID of the item. This is a hashed product identifier from Amazon.

* **overall** The rating of reviewer gave the item.

* **helpful** The helpfulness votes for the review. This has 2 subfields, 'nHelpful' and 'outOf'. The latter is the total number of votes this review received. The former is the number of those that considered the review to be helpful.

* **reviewText** The text of the review.

* **summary** The summary of the review.

* **unixReviewTime** The time of the review in seconds since 1970.

**meta.json** Contains metadata of the items:

* **asin** The ID of the item.

* **categories** The category labels of the item being reviewed.

* **price** The price of the item.

* **brand** The brand of the item.

**pairs_Rating.txt** The pairs (reviewerID and asin) on which you are to predict ratings.

**pairs_Purchase.txt** The pairs on which you are to predict whether a user purchased an item or not.

**pairs_Helpful.txt** The pairs on which you are to predict helpfulness votes. A third column in this file is the total number of votes from which you should predict how many were helpful.

**helpful.json** The review data associated with the helpfulness prediction test set. The 'nHelpful' field has been removed from this data since that is the value you need to predict above. This data will only be of use for the helpfulness prediction task.

**baseline.py** A simple baseline for each task.

## Tasks

**Rating prediction** Predict people's star ratings as accurately as possible for those (reviewerID, asin) pairs in 'pairs_Rating.txt'. Accuracy will be measured in terms of the [root mean-squared error (RMSE)](http://www.kaggle.com/wiki/RootMeanSquaredError).

**Purchase prediction** Predict given a (reviewerID, asin) pair from 'pairs_Purchase.txt' whether the user purchased the item (really, whether it was one of the items they reviewed). Accuracy will be measured in terms of the [categorization accuracy](http://www.kaggle.com/wiki/HammingLoss) (1 minus the Hamming loss).

**Helpfulness prediction** Predic whether a user's review of an item will be considered helpful. The file 'pairs_Helpful.txt' contains (reviewerID, asin) pairs with a third column containing the number of votes the user's review of the item received. You must predict how many of them were helpful. Accuracy will be measured in terms of the total [absolute error](http://www.kaggle.com/wiki/AbsoluteError), i.e. you are penalized one according to the difference |nHelpful - prediction|, where 'nHelpful' is the number of helpful votes the review actually received, and 'prediction' is your prediction of this quantity.

We set up competitions on Kaggle to keep track of your results compared to those of other members of the class. The leaderboard will show your results on half of the test data, but your ultimate score will depend on your predictions across the whole dataset.
* Kaggle competition: [rating prediction](https://inclass.kaggle.com/c/cs591-hw3-rating-prediction3) click here to [join](https://kaggle.com/join/datascience16rating)
* Kaggle competition: [purchase prediction](https://inclass.kaggle.com/c/cs591-hw3-purchase-prediction) click here to [join](https://kaggle.com/join/datascience16purchase)
* Kaggle competition: [helpfulness prediction](https://inclass.kaggle.com/c/cs591-hw3-helpful-prediction) click here to [join](https://kaggle.com/join/datascience16helpful)

## Grading and Evaluation

You will be graded on the following aspects.

* Your written report. This should describe the approaches you took to each of the 3 tasks. To obtain good performance, you should not need to invent new approaches (though you are more than welcome to) but rather you will be graded based on your decision to apply reasonable approaches to each of the given tasks. (**10pts** for each task)

* Your ability to obtain a solution which outperforms the baselines on the unseen portion of the test data. Obtaining full marks requires a solution which is substantially better (at least several percent) than baseline performance. (**10pts** for each task)

* Your ranking for each of the three tasks compared to other students in the class. (**5pts** for each task)

* Obtain a solution which outperforms the baselines on the seen portion of the test data (the leaderboard). 
(**5pts** for each task)

## Baselines

Simple baselines have been provided for each of the 3 tasks. These are included in 'baselines.py' among the files above. These 3 baselines operate as follows:

**Rating prediction** Returns the global average rating, or the user's average if you have seen them before in the training data.

**Purchase prediction** Finds the most popular products that account for 50% of purchases in the training data. Return '1' whenever such a product is seen at test time, '0' otherwise.

** Helpfulness prediction** Multiplies the number of votes by the global average helpfulness rate, or the user's rate if we saw this user in the training data.

Running 'baseline.py' produces 3 files containing predicted outputs. Your submission files should have the same format.

## Dataset Citation

**Image-based recommendations on styles and substitutes** J. McAuley, C. Targett, J. Shi, A. van den Hengel *SIGIR*, 2015

**Inferring networks of substitutable and complementary products** J. McAuley, R. Pandey, J. Leskovec *Knowledge Discovery and Data Mining*, 2015

### SOLUTION TASK 1:

For rating prediction:

Rating = Item Average + User Bias (while refering Simon Flunk's Netflix algorithm)
 
     Instead of taking Item average rating directly, I scaled it up using global average as well. Because if some item has a single rating, it doesn't necessarily mean it'll be the average rating. Similarly, I scaled user bias as well to get ratings with following approach:
     
                       Rating = scaled item average + scaled user bias
                        Kaggle score: 1.36615 (after trying various scaling parameters)         


### 1. Averaging:

In [33]:
# Libraries
import numpy as np
import pandas as pd

import gzip
from collections import defaultdict

import time
import sys
from scipy.stats import logistic


In [34]:
#parse data

def readJson(f):
  for l in open(f):
    yield eval(l)

now = time.time()
usersitem = defaultdict(dict) # (user_id,item_id) rating
mapui = defaultdict(list) # user_id, item_id
mapiu = defaultdict(list) # item_id, user_id

rate = {}

for l in readJson("train.json"):
    mapui[l['reviewerID']].append(l['asin'])
    mapiu[l['asin']].append(l['reviewerID'])
              
    usersitem[l['reviewerID']][l['asin']] = l['overall']

print "Done", len(usersitem), time.time()-now


Done 509678 92.3317990303


In [35]:
#parse category fields to be used in 3) regression models

now = time.time()
itemsattr = defaultdict(list) # item_id, category

attrset = set()

for l in readJson("meta.json"):
    categ = l['categories'].translate(None,'\'').split("[[")[1].split("]]")[0].split(",")[1:2]
    itemsattr[l['asin']] = categ
    attrset |= set(categ)
    
print "Done", time.time()-now, len(itemsattr), len(attrset)

Done 112.736366987 498196 42


In [36]:
#check entries
print itemsattr['03975a07fc9d5777a251e73cd7421aff026c7c5d3d58b7d66fae6d0b9d48ff7a']
print usersitem['bc19970fff3383b2fe947cf9a3a5d7b13b6e57ef2cd53abc52bb2dfedf5fb1cd']['19e5cc4a706554d37670eabca2c19f1fc4f259361d78f0b58dafb91f3a863fc1']

[' Car & Vehicle Electronics']
2.0


    Now we can start populating different types of averages and see what could work best. I calculated user's average ratings, item's average ratings, user's bias values and item's bias values.

In [37]:
#populate average ratings to get global averages and per-element averages

userfeature = defaultdict(dict)
itemfeature = defaultdict(dict)

avgratedict = defaultdict(list)
useroffdict = defaultdict()

start = time.time()

def populateAvgDict():
    for user in usersitem.keys():
        for item in usersitem[user].keys():
            #print user, usersitem[user], item, itemsattr[item]
            avgratedict[item].append(usersitem[user][item])
            #for feature in list(attrset):
            for feature in itemsattr[item]:
                #if feature in itemsattr[item]:
                userfeature[user][feature] = usersitem[user][item]
                itemfeature[item][feature] = usersitem[user][item]
                """
                else:
                    userfeature[user][feature] = 0.0
                    itemfeature[item][feature] = 0.0
                """

                """
                if len(userfeature) == 5:
                    print userfeature, itemfeature
                    sys.exit(0)
                """    
    return avgratedict
        
avgratedict = populateAvgDict()  
print len(userfeature), len(itemfeature), time.time()-start

509678 171185 4.8681499958


In [38]:
#check entries

print userfeature['bc19970fff3383b2fe947cf9a3a5d7b13b6e57ef2cd53abc52bb2dfedf5fb1cd']
print itemfeature['03975a07fc9d5777a251e73cd7421aff026c7c5d3d58b7d66fae6d0b9d48ff7a']


{' Computers & Accessories': 2.0, ' Television & Video': 4.0, ' Accessories & Supplies': 2.0}
{' Car & Vehicle Electronics': 3.0}


In [39]:
#get global average of item ratings in "globalavg"

globalsum = 0
globalsize = 0

for k,v in avgratedict.iteritems():
    globalsum += sum(v)
    globalsize += len(v)
    
globalavg = globalsum/globalsize

print globalsum, globalsize, globalavg


3837538.0 1000000 3.837538


    Most of the algorithms I've seen take scaling factors (lambdas in BellKor) based on V_a and V_b, where V_a is global variance of item's ratings and V_b is variance between per item's average rating and it's actual rating

In [40]:
#calculate variance Va and Vb - this affects which parameter to choose for scaling

rateVa = 0.0
rateVb = 0.0

for k,v in avgratedict.iteritems():
    temprateVb = 0.0
    if not len(v):
        print k
        break
    curavg = 1.0*sum(v)/len(v)
    for r in v:
        rateVa += (globalavg-r)**2
        rateVb += (curavg-r)**2

print rateVa, rateVb, (rateVb/len(avgratedict))/(rateVa/globalsize)

2111356.09854 1538264.88976 4.25602267766


In [41]:
#get user's bias values

useroffset = defaultdict(list)
gloffsetsum = 0.0
gloffsetlen = 0
for user,items in usersitem.iteritems():
    for item, rating in items.iteritems():        
        off = usersitem[user][item] -  sum(avgratedict[item])/len(avgratedict[item])
        useroffset[user].append(off)
        gloffsetsum += off
        gloffsetlen+=1
        
gloffsetavg = gloffsetsum/gloffsetlen
print "Computed users' offset", len(useroffset), gloffsetsum, gloffsetlen
     

Computed users' offset 509678 -7.67386154621e-11 1000000


In [42]:
#get item's bias values - NOT chosen for final prediction routine

itemoffset = defaultdict(list)
itemgloffsetsum = 0.0
itemgloffsetlen = 0
for items,ratelist in avgratedict.iteritems():
    for rating in ratelist:
        off = rating - sum(avgratedict[item])/len(avgratedict[item])
        itemoffset[items].append(off)
        
        #off = usersitem[user][item] -  sum(avgratedict[item])/len(avgratedict[item])
        #useroffset[user].append(off)
        
        itemgloffsetsum += off
        itemgloffsetlen+=1
        
itemgloffsetavg = itemgloffsetsum/itemgloffsetlen
print "Computed items' offset", len(itemoffset), itemgloffsetsum, itemgloffsetlen
     

Computed items' offset 171185 233142.395601 1000000


In [43]:
#calculate variance Va and Vb for offset - reference for scaling parameters

offVa = 0.0
offVb = 0.0

for user,items in usersitem.iteritems():
    userOff = 0.0
    userOffAvg = sum(useroffset[user] )/len(useroffset[user])
    
    for item, rating in items.iteritems():        
        curOff = usersitem[user][item] -  sum(avgratedict[item])/len(avgratedict[item])
        offVa += (gloffsetavg - curOff)**2
        userOff += (userOffAvg - curOff)**2
    offVb += userOff
        

print offVa, offVb, (offVb/len(useroffset))/(rateVa/gloffsetlen)

1538264.88976 591856.023241 0.549994943614


    Instead of reading test data again and again, store it in memory

In [44]:
def readJson(f):
  for l in open(f):
    yield eval(l)

def parseData(fname):
  for l in urllib.urlopen(fname):
    yield eval(l)
    
allRatings = []
userRatings = defaultdict(list)
data = []

for l in readJson("train.json"):
    data.append(l)

In [ ]:
#Train average ratings and user bias values to get better predictions

itemavgdict = defaultdict()
userbiasdict = defaultdict()
itembiasdict = defaultdict()

def SVDBaseTrain(userid, itemid, actual, k1=6.5, k2=3.5):
    """
    Take user-item pair and it's actual rating. Compare with the predicted rating and adjust 
    average rating and bias ratings
    """
    ret = 0.0
    try:
        rateavg = avgratedict[itemid]
    except KeyError:
        rateavg = []
    try:
        offset = useroffset[userid] 
    except KeyError:
        offset = []
        
    try:
        itemoff = itemoffset[itemid]
    except KeyError:
        itemoff = []

    itemavg = 0.0
    offsetavg = 0.0
    itembiasavg = 0.0
    
    for i in xrange(5):
        
        """
        Scaling logic:
        value = (K* value's global avg + global sum)/(K + global length)
        This K depends on V_a and V_b of value and I found 6.5 and 3.5 to be sufficiently good
        """
        newitemavg = (globalavg*k1 + sum(rateavg))/(k1 + len(rateavg))
        newoffsetavg = (gloffsetavg*k2+sum(offset))/(k2+len(offset))
        newitemoff = (itemgloffsetavg*k2+sum(itemoff))/(k2+len(itemoff))
        
        trainpredict = newitemavg + newoffsetavg
        err = actual- trainpredict
        #if err < 0.001:
            #break
        itemavg = newitemavg + 0.001*(err) #+ 0.0001*(err**2)
        offsetavg = newoffsetavg + 0.001*(err) #+ 0.0001*(err**2)
             
        itembiasavg = newitemoff + 0.0002*(err)
        
    itemavgdict[itemid] = itemavg
    userbiasdict[userid] = offsetavg
    itembiasdict[itemid] = itembiasavg
    
    ret =  min(4.6, max(1.6,itemavg+offsetavg))
    return ret


mse = 0.0
for test in data:
    actual = test['overall']
    predict = SVDBaseTrain(test['reviewerID'], test['asin'], actual)
    
    mse += (predict-actual)**2
    #break
print mse



1233948.17301


    Get predictions based on trained data and write to file (0.4 sec for 100,000 pairs - very fast)

In [ ]:
import math


def SVDBase(userid, itemid, k1=6.5, k2=3.5):
    """
    This is essentially same logic as used in training.    
    """
    ret = 0.0
    try:
        itemavg = itemavgdict[itemid]
    except:
    
        try:
            rateavg = avgratedict[itemid]
        except KeyError:
            rateavg = []
        itemavg = (globalavg*k1 + sum(rateavg))/(k1 + len(rateavg))
    try:
        offsetavg = userbiasdict[userid]
    except:
        try:
            offset = useroffset[userid] 
        except KeyError:
            offset = []
        offsetavg = (gloffsetavg*k2+sum(offset))/(k2+len(offset))
            
    try:
        itembias = itembiasdict[itemid]
    except KeyError:
        try:
            itemoff = itemoffset[itemid]
        except KeyError:
            itemoff = []
        itembias = (itemgloffsetavg*k2+sum(itemoff))/(k2+len(itemoff))*0.5
        
        
          
    ret =  min(4.6, max(1.6,itemavg+offsetavg))
     
    return ret


ratefile = open("biasTrain_Rating.txt","w")
#intratefile = open("INTpreSVD_Rating.txt","w")
ratefile.write("reviewerID-asin,prediction\n")
#intratefile.write("reviewerID-asin,prediction\n")
start = time.time()

reqUser = defaultdict(dict)
reqItem = defaultdict(dict)
reqAttr = set()

with open("pairs_Rating.txt") as prate:
    for pair in prate:
        #print pair
        if pair.startswith("reviewer"):
            continue
        
        userid, itemid = pair.split("-")
        itemid = itemid[:-1]
        basepredict = SVDBase(userid,itemid)                
                
        ratefile.write("%s-%s,%f\n" % (userid, itemid, basepredict))
        #intratefile.write("%s-%s,%d\n" % (userid, itemid, int(round(basepredict))))
        #sys.exit(0)
ratefile.close()
#intratefile.close()
        
print "Done", time.time()-start

    Following cell is just a backup of my previous attempt

In [ ]:
#backup

import math
#head start before SVD


def SVDBase(userid, itemid, k1=4, k2=4):
    ret = 0.0
    try:
        rateavg = avgratedict[itemid]
    except KeyError:
        rateavg = []
    try:
        offset = useroffset[userid] 
    except KeyError:
        offset = []
        
    #if not len(rateavg) or not len(offset):
        #print userid, itemid
    #if len(rateavg):
    itemavg = (globalavg*k1 + sum(rateavg))/(k1 + len(rateavg))
    #else:
        #itemavg = globalavg
    #if len(offset):
    offsetavg = (gloffsetavg*k2+sum(offset))/(k2+len(offset))
    #else:
        #offsetavg = 0.0

    ret =  min(5.0, max(0.5,itemavg+offsetavg))

    #except KeyError:
        #ret = min(5.0, max(0.5, globalavg))
        
    return ret


ratefile = open("preSVDpred_Rating.txt","w")
#intratefile = open("INTpreSVD_Rating.txt","w")
ratefile.write("reviewerID-asin,prediction\n")
#intratefile.write("reviewerID-asin,prediction\n")
start = time.time()

reqUser = defaultdict(dict)
reqItem = defaultdict(dict)
reqAttr = set()

with open("pairs_Rating.txt") as prate:
    for pair in prate:
        #print pair
        if pair.startswith("reviewer"):
            continue
        
        userid, itemid = pair.split("-")
        itemid = itemid[:-1]
        basepredict = SVDBase(userid,itemid)                
        
        reqUser[userid][itemid] = basepredict
        reqItem[itemid][userid] = basepredict
        reqAttr |= set(itemsattr[itemid])
        usersitem[userid][itemid] = basepredict
        ratefile.write("%s-%s,%f\n" % (userid, itemid, basepredict))
        #intratefile.write("%s-%s,%d\n" % (userid, itemid, int(round(basepredict))))
        #sys.exit(0)
ratefile.close()
#intratefile.close()
        
print "Done", time.time()-start, len(reqUser), len(reqItem), len(reqAttr)

------------

-----------------